# Scrape and add most recent seasons to all time results

In [29]:
## Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import logging
from tqdm import tqdm
import re
import os



### Parse the current season schedule / results page

def parse_current_season(url):
        # Initialize variables
    current_date = None
    current_conference = None
    game_notes = None

    # Initialize an empty list to hold the data
    data = []

    # Parse the page with BeautifulSoup
    # Get the page with requests
    response = requests.get(url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # select the table or tables
    tables = soup.find_all('table')

    rows = soup.find_all('tr')

    # Loop through each row to find relevant information
    for row in rows:
        # Check for date row
        if row.get('class') == ['stats-section']:
            current_date = row.find('td').text.strip()
        # Check for conference row
        elif row.get('class') == ['sked-header']:
            current_conference = row.find('td').text.strip()
        # Check for game notes
        elif len(row.find_all('td')) == 2:
            game_notes = row.find_all('td')[1].text.strip()
        # Process rows with game data
        elif row.get('valign') == 'top':
            cells = row.find_all('td')
            if len(cells) >= 9:
                home_team = cells[0].text.strip()
                # Remove any hyphens from the team name
                home_team = home_team.replace('-', ' ')
                home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
                home_score = cells[1].text.strip()
                away_team = cells[3].text.strip()
                away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
                away_score = cells[4].text.strip()
                ot = cells[5].text.strip()
                box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
                metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
                # Capture Game Notes
                game_notes_cell = cells[-1].find('small')
                game_notes = game_notes_cell.text.strip() if game_notes_cell else None

                # Append data to the list
                data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
                game_notes = None  # Reset game notes for the next row
    return data


### Set The Target Year
# current_year_url = "https://www.collegehockeynews.com/schedules/?season=20242025" # 2024-2025 season
current_year_url = "https://www.collegehockeynews.com/schedules/?season=20232024" # 2023-2024 season
## call the function
data = parse_current_season(current_year_url)


# Create a dataframe from the list

columns = ['Date', 'Conference', 'Game_Notes', 'Home_Team', 'Home_Team_Link', 'Home_Score', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)
            
## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# remove day of the week from date
# format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

### Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Loop to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    # Remove any hyphens from the team name if there are any
    home_team_abbr = home_team_abbr.replace('-', ' ')
    away_team_abbr = away_team_abbr.replace('-', ' ')
    game_id = f'{row.Date}-{home_team_abbr}-{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Remove any hyphens from the team names if any
df['Home_Team'] = df['Home_Team'].str.replace('-', ' ')
df['Away_Team'] = df['Away_Team'].str.replace('-', ' ')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}-{row.Home_Team}-{row.Away_Team}', axis=1)

## Filter out games that have not been played yet
df = df[df['Home_Score'] != '']

# Replace Nan values in metrics column with empty string
df['Metrics_Link'] = df['Metrics_Link'].fillna('')

In [30]:
#### INSPECT RESULTING DATAFRAME

df.head()



,Date,Conference,Game_Notes,Home_Team,Home_Team_Link,Home_Score,Away_Team,Away_Team_Link,Away_Score,OT,Box_Link,Metrics_Link,Day,Game_ID
0,2023-10-06,Exhibition,,Arizona,None,0,Arizona State,/reports/team/Arizona-State/61,16,,None,,Friday,2023-10-06-Arizona-Arizona State
1,2023-10-07,Exhibition,,McGill,None,0,Vermont,/reports/team/Vermont/55,4,,None,,Saturday,2023-10-07-McGill-Vermont
2,2023-10-07,Exhibition,,Merrimack,/reports/team/Merrimack/29,3,Sacred Heart,/reports/team/Sacred-Heart/51,2,ot,None,,Saturday,2023-10-07-Merrimack-Sacred Heart
3,2023-10-07,Exhibition,,Union,/reports/team/Union/54,2,Rensselaer,/reports/team/Rensselaer/48,2,ot,None,,Saturday,2023-10-07-Union-Rensselaer
4,2023-10-07,Exhibition,,Royal Military,None,3,Niagara,/reports/team/Niagara/39,8,,None,,Saturday,2023-10-07-Royal Military-Niagara


In [31]:
# Rename to df_2024
# df_2024 = df.copy()
df_2023 = df.copy()



In [32]:
# Show the resulting DataFrames 2023 and 2024 and combine
# df_2023.head(), df_2024.head()

df_new = pd.concat([df_2023, df_2024], axis=0)



In [33]:
# load all time table thru 2023
df_thru_2023 = pd.read_csv("../data/tables/results_all_time_thru_2023.csv")

# check the dataframe
df_thru_2023.tail()


,Date,Conference,Home_Team,Home_Team_Link,Home_Score,Away_Team,Away_Team_Link,Away_Score,OT,Game_Notes,Game_ID,Day,Season_Year
53647,3/26/2023,"NCAA Tournament, Bridgeport Regional (at Total...",Ohio State,/reports/team/Ohio-State/44,1,Quinnipiac,/reports/team/Quinnipiac/47,4,NaN,Bridgeport Regional Final,2023-03-26_Ohio State_Quinnipiac,Sunday,2022
53648,3/26/2023,"NCAA Tournament, Allentown Regional (at PPL Ce...",Penn State,/reports/team/Penn-State/60,1,Michigan,/reports/team/Michigan/31,2,ot,Allentown Regional Final,2023-03-26_Penn State_Michigan,Sunday,2022
53649,4/6/2023,"NCAA Tournament (at Amalie Arena, Tampa, Fla.)",Boston University,/reports/team/Boston-University/10,2,Minnesota,/reports/team/Minnesota/34,6,NaN,National Semifinal,2023-04-06_Boston University_Minnesota,Thursday,2022
53650,4/6/2023,"NCAA Tournament (at Amalie Arena, Tampa, Fla.)",Michigan,/reports/team/Michigan/31,2,Quinnipiac,/reports/team/Quinnipiac/47,5,NaN,National Semifinal,2023-04-06_Michigan_Quinnipiac,Thursday,2022
53651,4/8/2023,"NCAA Tournament (at Amalie Arena, Tampa, Fla.)",Quinnipiac,/reports/team/Quinnipiac/47,3,Minnesota,/reports/team/Minnesota/34,2,ot,National Championship,2023-04-08_Quinnipiac_Minnesota,Saturday,2022


In [34]:
# add the new dataframe into the all time dataframe
df_all_time = pd.concat([df_thru_2023, df_new], axis=0)

# Check and normalize the column data types
df_all_time = df_all_time.convert_dtypes()

# save to CSV
df_all_time.to_csv("../data/tables/results_all_time.csv", index=False)

In [35]:
df_all_time.tail()

,Date,Conference,Home_Team,Home_Team_Link,Home_Score,Away_Team,Away_Team_Link,Away_Score,OT,Game_Notes,Game_ID,Day,Season_Year,Box_Link,Metrics_Link
1225,2025-03-30,"NCAA Tournament, Allentown Regional (at PPL Ce...",Penn State,/reports/team/Penn-State/60,3,Connecticut,/reports/team/Connecticut/17,2,ot,Allentown Regional Final,2025-03-30-Penn State-Connecticut,Sunday,<NA>,/box/final/20250330/psu/con/,/box/metrics.php?gd=103968
1226,2025-03-30,"NCAA Tournament, Manchester Regional (at SNHU ...",Boston College,/reports/team/Boston-College/9,1,Denver,/reports/team/Denver/20,3,,Manchester Regional Final,2025-03-30-Boston College-Denver,Sunday,<NA>,/box/final/20250330/bc_/den/,/box/metrics.php?gd=103977
1227,2025-04-10,"NCAA Tournament (at Enterprise Center, St. Lou...",Denver,/reports/team/Denver/20,2,Western Michigan,/reports/team/Western-Michigan/57,3,2ot,National Semifinal,2025-04-10-Denver-Western Michigan,Thursday,<NA>,/box/final/20250410/den/wmu/,/box/metrics.php?gd=103978
1228,2025-04-10,"NCAA Tournament (at Enterprise Center, St. Lou...",Boston University,/reports/team/Boston-University/10,3,Penn State,/reports/team/Penn-State/60,1,,National Semifinal,2025-04-10-Boston University-Penn State,Thursday,<NA>,/box/final/20250410/bu_/psu/,/box/metrics.php?gd=103979
1229,2025-04-12,"NCAA Tournament (at Enterprise Center, St. Lou...",Western Michigan,/reports/team/Western-Michigan/57,6,Boston University,/reports/team/Boston-University/10,2,,National Championship,2025-04-12-Western Michigan-Boston University,Saturday,<NA>,/box/final/20250412/wmu/bu_/,/box/metrics.php?gd=103980


# Below FIRST DRAFT - POC - All Time Games above below 500

In [36]:
import re
import numpy as np
import pandas as pd
import os

# --- 1) Load & normalize ---

all_time_path = os.path.join("..", "data", "tables", "results_all_time_thru_2023.csv")
DATA_PATH = all_time_path

df = pd.read_csv(all_time_path)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Home_Score'] = pd.to_numeric(df['Home_Score'], errors='coerce').astype('Int64')
df['Away_Score'] = pd.to_numeric(df['Away_Score'], errors='coerce').astype('Int64')
df['OT'] = df['OT'].astype(str).str.lower().replace({'nan':''})
df['Game_Notes'] = df['Game_Notes'].astype(str).str.lower()

def extract_id(s):
    m = re.search(r"/reports/team/.+?/(\d+)", str(s))
    return int(m.group(1)) if m else pd.NA

df['Home_Team_ID'] = df['Home_Team_Link'].apply(extract_id)
df['Away_Team_ID'] = df['Away_Team_Link'].apply(extract_id)

# --- 2) Filter what counts ---
mask_both_teams = df['Home_Team_ID'].notna() & df['Away_Team_ID'].notna()
mask_exhib = df['Game_Notes'].str.contains('exhib', na=False)
mask_cancel = df['Game_Notes'].str.contains('cancel', na=False)
df = df[mask_both_teams & ~mask_exhib & ~mask_cancel].copy()

# --- 3) Explode to long ---
home = df.rename(columns={
    'Home_Team':'team','Home_Team_ID':'team_id','Home_Score':'team_score',
    'Away_Team':'opp','Away_Team_ID':'opp_id','Away_Score':'opp_score'
}).assign(venue='H')
away = df.rename(columns={
    'Away_Team':'team','Away_Team_ID':'team_id','Away_Score':'team_score',
    'Home_Team':'opp','Home_Team_ID':'opp_id','Home_Score':'opp_score'
}).assign(venue='A')

long = pd.concat([home, away], ignore_index=True)

keep_cols = ['Date','Season_Year','Game_ID','team','team_id','opp','opp_id',
             'team_score','opp_score','venue','OT','Conference','Game_Notes']
long = long[keep_cols].copy()

# --- 4) Outcome & increment ---
long['result'] = np.select(
    [long['team_score'] > long['opp_score'],
     long['team_score'] < long['opp_score']],
    ['W','L'],
    default='T'
)
long['is_win']  = (long['result']=='W').astype(int)
long['is_loss'] = (long['result']=='L').astype(int)
long['is_tie']  = (long['result']=='T').astype(int)

# For the .500 curve: ties are neutral
long['delta'] = long['is_win'] - long['is_loss']  # +1 / -1 / 0

# --- 5) Sort & sequence ---
long = long.sort_values(['team_id','Date','Game_ID'], kind='mergesort')
long['gp'] = long.groupby('team_id').cumcount() + 1

# --- 6) Cumulative record & games above .500 ---
long['cum_w'] = long.groupby('team_id')['is_win'].cumsum()
long['cum_l'] = long.groupby('team_id')['is_loss'].cumsum()
long['cum_t'] = long.groupby('team_id')['is_tie'].cumsum()
long['games_above_500'] = long['cum_w'] - long['cum_l']
long['record_str'] = long['cum_w'].astype(str)+'-'+long['cum_l'].astype(str)+'-'+long['cum_t'].astype(str)

timeline = long[['team_id','team','Date','Game_ID','Season_Year','gp',
                 'result','delta','cum_w','cum_l','cum_t','games_above_500']].copy()

# --- 7) Milestones for annotations ---
def team_summary(g):
    g = g.sort_values('Date')
    max_idx = g['games_above_500'].idxmax()
    min_idx = g['games_above_500'].idxmin()
    out = {
        'team': g['team'].iloc[0],
        'first_date': g['Date'].min(),
        'last_date': g['Date'].max(),
        'max_point': g.loc[max_idx, 'games_above_500'],
        'max_date': g.loc[max_idx, 'Date'],
        'min_point': g.loc[min_idx, 'games_above_500'],
        'min_date': g.loc[min_idx, 'Date'],
        'final_w': g['cum_w'].iloc[-1],
        'final_l': g['cum_l'].iloc[-1],
        'final_t': g['cum_t'].iloc[-1],
    }
    # last time exactly at .500 (if ever)
    at500 = g[g['games_above_500']==0]
    out['last_500_date'] = at500['Date'].iloc[-1] if not at500.empty else pd.NaT
    return pd.Series(out)

summary = timeline.groupby('team_id').apply(team_summary).reset_index()

# --- 8) Save for plotting ---
timeline.to_parquet('team_timeseries.parquet', index=False)
summary.to_parquet('team_summary.parquet', index=False)
# (or .csv if you prefer)


C:\Users\jbanc\AppData\Local\Temp\ipykernel_22516\2933784045.py:97: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = timeline.groupby('team_id').apply(team_summary).reset_index()


In [37]:
# Re-run full pipeline (concise version) and save outputs, avoiding any UI display calls.

import re, pandas as pd, numpy as np
from pathlib import Path

# DATA_PATH = Path("/mnt/data/results_table_all_time_thru 2023.csv")
df = pd.read_csv(DATA_PATH)

def find_col(possible, cols):
    norm = {c.lower().replace(" ", "_"): c for c in cols}
    for p in possible:
        key = p.lower().replace(" ", "_")
        if key in norm: return norm[key]
    return None

cols = df.columns.tolist()
c_date   = find_col(["Date"], cols)
c_season = find_col(["Season_Year","Season","Season Year"], cols)
c_gmid   = find_col(["Game_ID","GameID","Game Id","gid"], cols)
c_htm    = find_col(["Home_Team","Home Team","Home"], cols)
c_atm    = find_col(["Away_Team","Away Team","Away","Visitor_Team","Visitor Team"], cols)
c_hsc    = find_col(["Home_Score","Home Score","HomeGoals","Home_Goals"], cols)
c_asc    = find_col(["Away_Score","Away Score","AwayGoals","Away_Goals"], cols)
c_ot     = find_col(["OT","OT_SO","OT/SO","Overtime","Shootout"], cols)
c_notes  = find_col(["Game_Notes","Notes","Game Notes"], cols)
c_hlnk   = find_col(["Home_Team_Link","Home Team Link","Home_Link"], cols)
c_alnk   = find_col(["Away_Team_Link","Away Team Link","Away_Link"], cols)

df[c_date] = pd.to_datetime(df[c_date], errors="coerce")
if c_season is None:
    c_season="__Season_Fallback__"; df[c_season]=df[c_date].dt.year
if c_gmid is None:
    c_gmid="__GameID_Fallback__"; df[c_gmid]=np.arange(1,len(df)+1)
if c_hsc: df[c_hsc]=pd.to_numeric(df[c_hsc], errors="coerce").astype("Int64")
if c_asc: df[c_asc]=pd.to_numeric(df[c_asc], errors="coerce").astype("Int64")
if c_ot: df[c_ot]=df[c_ot].astype(str).str.lower()
if c_notes: df[c_notes]=df[c_notes].astype(str).str.lower()

def extract_team_id(u):
    if pd.isna(u): return pd.NA
    m=re.search(r"/reports/team/.+?/(\d+)", str(u))
    return int(m.group(1)) if m else pd.NA

if c_hlnk and c_alnk:
    df["Home_Team_ID"]=df[c_hlnk].apply(extract_team_id)
    df["Away_Team_ID"]=df[c_alnk].apply(extract_team_id)
else:
    df["Home_Team_ID"]=df[c_htm].astype(str).str.strip().str.lower().map(lambda s: abs(hash(s))%(10**9))
    df["Away_Team_ID"]=df[c_atm].astype(str).str.strip().str.lower().map(lambda s: abs(hash(s))%(10**9))

mask_valid = df["Home_Team_ID"].notna() & df["Away_Team_ID"].notna()
mask_exhib = df[c_notes].str.contains("exhib", na=False) if c_notes else False
mask_cancel= df[c_notes].str.contains("cancel",na=False) if c_notes else False
df = df[mask_valid & ~mask_exhib & ~mask_cancel].copy()

home = df.rename(columns={
    c_date:"Date", c_season:"Season_Year", c_gmid:"Game_ID",
    c_htm:"team","Home_Team_ID":"team_id", c_hsc:"team_score",
    c_atm:"opp","Away_Team_ID":"opp_id", c_asc:"opp_score",
    **({c_ot:"OT"} if c_ot else {}), **({c_notes:"Game_Notes"} if c_notes else {})
})[["Date","Season_Year","Game_ID","team","team_id","team_score","opp","opp_id","opp_score"] + (["OT"] if c_ot else []) + (["Game_Notes"] if c_notes else [])]
home["venue"]="H"

away = df.rename(columns={
    c_date:"Date", c_season:"Season_Year", c_gmid:"Game_ID",
    c_atm:"team","Away_Team_ID":"team_id", c_asc:"team_score",
    c_htm:"opp","Home_Team_ID":"opp_id", c_hsc:"opp_score",
    **({c_ot:"OT"} if c_ot else {}), **({c_notes:"Game_Notes"} if c_notes else {})
})[["Date","Season_Year","Game_ID","team","team_id","team_score","opp","opp_id","opp_score"] + (["OT"] if c_ot else []) + (["Game_Notes"] if c_notes else [])]
away["venue"]="A"

long=pd.concat([home,away], ignore_index=True)
long=long[long["team_score"].notna() & long["opp_score"].notna()].copy()

long["result"]=np.where(long["team_score"]>long["opp_score"],"W",np.where(long["team_score"]<long["opp_score"],"L","T"))
long["is_win"]=(long["result"]=="W").astype(int)
long["is_loss"]=(long["result"]=="L").astype(int)
long["is_tie"]=(long["result"]=="T").astype(int)
long=long.sort_values(["team_id","Date","Game_ID"], kind="mergesort")
long["gp"]=long.groupby("team_id").cumcount()+1
long["cum_w"]=long.groupby("team_id")["is_win"].cumsum()
long["cum_l"]=long.groupby("team_id")["is_loss"].cumsum()
long["cum_t"]=long.groupby("team_id")["is_tie"].cumsum()
long["games_above_500"]=long["cum_w"]-long["cum_l"]
long["delta"]=long["is_win"]-long["is_loss"]

timeline=long[["team_id","team","Date","Game_ID","Season_Year","gp","result","delta","cum_w","cum_l","cum_t","games_above_500","venue"]].copy()

def summarize(g):
    g=g.sort_values("Date")
    at500=g[g["games_above_500"]==0]
    return pd.Series({
        "team":g["team"].iloc[0],
        "first_date":g["Date"].min(),
        "last_date":g["Date"].max(),
        "max_point":g["games_above_500"].max(),
        "max_date":g.loc[g["games_above_500"].idxmax(),"Date"],
        "min_point":g["games_above_500"].min(),
        "min_date":g.loc[g["games_above_500"].idxmin(),"Date"],
        "final_w":g["cum_w"].iloc[-1],
        "final_l":g["cum_l"].iloc[-1],
        "final_t":g["cum_t"].iloc[-1],
        "final_games_above_500":g["games_above_500"].iloc[-1],
        "last_500_date": at500["Date"].iloc[-1] if not at500.empty else pd.NaT,
    })

summary = timeline.groupby("team_id", group_keys=False).apply(summarize).reset_index()

out_dir=Path("../TEMP")
timeline.to_parquet(out_dir/"team_timeseries.parquet", index=False)
summary.to_parquet(out_dir/"team_summary.parquet", index=False)
timeline.to_csv(out_dir/"team_timeseries.csv", index=False)
summary.to_csv(out_dir/"team_summary.csv", index=False)

print("Done. Files created in /mnt/data:")
print("- team_timeseries.parquet  |  team_timeseries.csv")
print("- team_summary.parquet     |  team_summary.csv")


C:\Users\jbanc\AppData\Local\Temp\ipykernel_22516\1220221622.py:107: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = timeline.groupby("team_id", group_keys=False).apply(summarize).reset_index()


Done. Files created in /mnt/data:
- team_timeseries.parquet  |  team_timeseries.csv
- team_summary.parquet     |  team_summary.csv


In [38]:
"""
transform_games_to_team_timeseries.py

Purpose
-------
Turn a "one row per game" college hockey CSV into:
  1) a per-team, per-game time series with cumulative record and games-above-.500
  2) a per-team summary for annotations (max/min points, last .500 date, final record)

Key Decisions
-------------
- .500 curve uses the classic definition: +1 for a WIN, -1 for a LOSS, 0 for a TIE.
- Ties do not move the line (they still appear in the cumulative record).
- Exhibitions / cancellations are excluded if noted in the data.
- If official team link IDs are missing, a stable hash of team names is used for `team_id`.

Output
------
- team_timeseries.csv  : one row per team per game with 'games_above_500'
- team_summary.csv     : per-team milestones for chart annotations

Author: you + ChatGPT
"""

import re
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Optional, List

# ---------- Utilities ---------------------------------------------------------

def find_col(possible: List[str], cols: List[str]) -> Optional[str]:
    """
    Fuzzy column resolver: try common alternatives, tolerant to spaces/underscores/case.
    Returns the first match found or None.
    """
    norm = {c.lower().replace(" ", "_"): c for c in cols}
    for p in possible:
        key = p.lower().replace(" ", "_")
        if key in norm:
            return norm[key]
    return None

def extract_team_id(url_like) -> Optional[int]:
    """
    If your source has official team links, extract the numeric ID from a pattern like:
      '/reports/team/<something>/<id>'
    Returns NA if no match.
    """
    if pd.isna(url_like):
        return pd.NA
    m = re.search(r"/reports/team/.+?/(\d+)", str(url_like))
    return int(m.group(1)) if m else pd.NA


# ---------- Main transformation ----------------------------------------------

def build_timeseries(input_csv: str, out_dir: str = "./") -> None:
    out_path = Path(out_dir)
    out_path.mkdir(parents=True, exist_ok=True)

    # 1) Load & normalize ------------------------------------------------------
    df_raw = pd.read_csv(input_csv)
    cols = df_raw.columns.tolist()

    c_date   = find_col(["Date"], cols)
    c_season = find_col(["Season_Year","Season","Season Year"], cols)
    c_gmid   = find_col(["Game_ID","GameID","Game Id","gid"], cols)

    c_htm    = find_col(["Home_Team","Home Team","Home"], cols)
    c_atm    = find_col(["Away_Team","Away Team","Away","Visitor_Team","Visitor Team"], cols)
    c_hsc    = find_col(["Home_Score","Home Score","HomeGoals","Home_Goals"], cols)
    c_asc    = find_col(["Away_Score","Away Score","AwayGoals","Away_Goals"], cols)

    c_ot     = find_col(["OT","OT_SO","OT/SO","Overtime","Shootout"], cols)
    c_notes  = find_col(["Game_Notes","Notes","Game Notes"], cols)

    c_hlnk   = find_col(["Home_Team_Link","Home Team Link","Home_Link"], cols)
    c_alnk   = find_col(["Away_Team_Link","Away Team Link","Away_Link"], cols)

    if c_date is None:
        raise ValueError("No 'Date' column found—cannot build timelines.")

    df = df_raw.copy()
    df[c_date] = pd.to_datetime(df[c_date], errors="coerce")

    if c_season is None:
        # Simple fallback: use calendar year of the date.
        c_season = "__Season_Fallback__"
        df[c_season] = df[c_date].dt.year

    if c_gmid is None:
        # Stable fallback game_id if not supplied
        c_gmid = "__GameID_Fallback__"
        df[c_gmid] = np.arange(1, len(df) + 1)

    # Scores
    for score_col in [c_hsc, c_asc]:
        if score_col is not None:
            df[score_col] = pd.to_numeric(df[score_col], errors="coerce").astype("Int64")

    # Text normalization
    if c_ot is not None:
        df[c_ot] = df[c_ot].astype(str).str.lower()
    if c_notes is not None:
        df[c_notes] = df[c_notes].astype(str).str.lower()

    # 2) Team IDs --------------------------------------------------------------
    # Prefer official IDs from link columns; otherwise, make a stable hash of team names.
    if c_hlnk and c_alnk:
        df["Home_Team_ID"] = df[c_hlnk].apply(extract_team_id)
        df["Away_Team_ID"] = df[c_alnk].apply(extract_team_id)
    else:
        df["Home_Team_ID"] = df[c_htm].astype(str).str.strip().str.lower().map(lambda s: abs(hash(s)) % (10**9))
        df["Away_Team_ID"] = df[c_atm].astype(str).str.strip().str.lower().map(lambda s: abs(hash(s)) % (10**9))

    # 3) Filter what shouldn’t count ------------------------------------------
    mask_valid = df["Home_Team_ID"].notna() & df["Away_Team_ID"].notna()
    if c_notes is not None:
        mask_exhib  = df[c_notes].str.contains("exhib",  na=False)
        mask_cancel = df[c_notes].str.contains("cancel", na=False)
    else:
        mask_exhib = mask_cancel = False

    df = df[mask_valid & ~mask_exhib & ~mask_cancel].copy()

    # 4) Explode to long (one row per team per game) --------------------------
    home = df.rename(columns={
        c_date:"Date", c_season:"Season_Year", c_gmid:"Game_ID",
        c_htm:"team", "Home_Team_ID":"team_id", c_hsc:"team_score",
        c_atm:"opp", "Away_Team_ID":"opp_id", c_asc:"opp_score",
        **({c_ot:"OT"} if c_ot else {}), **({c_notes:"Game_Notes"} if c_notes else {}),
    })
    home = home[["Date","Season_Year","Game_ID","team","team_id","team_score","opp","opp_id","opp_score"] +
                (["OT"] if c_ot else []) + (["Game_Notes"] if c_notes else [])]
    home["venue"] = "H"

    away = df.rename(columns={
        c_date:"Date", c_season:"Season_Year", c_gmid:"Game_ID",
        c_atm:"team", "Away_Team_ID":"team_id", c_asc:"team_score",
        c_htm:"opp", "Home_Team_ID":"opp_id", c_hsc:"opp_score",
        **({c_ot:"OT"} if c_ot else {}), **({c_notes:"Game_Notes"} if c_notes else {}),
    })
    away = away[["Date","Season_Year","Game_ID","team","team_id","team_score","opp","opp_id","opp_score"] +
                (["OT"] if c_ot else []) + (["Game_Notes"] if c_notes else [])]
    away["venue"] = "A"

    long = pd.concat([home, away], ignore_index=True)

    # Drop rows without scores (postponed/unplayed/etc.)
    long = long[long["team_score"].notna() & long["opp_score"].notna()].copy()

    # 5) Outcome & increment for the .500 curve --------------------------------
    long["result"]  = np.where(long["team_score"] > long["opp_score"], "W",
                        np.where(long["team_score"] < long["opp_score"], "L", "T"))
    long["is_win"]  = (long["result"] == "W").astype(int)
    long["is_loss"] = (long["result"] == "L").astype(int)
    long["is_tie"]  = (long["result"] == "T").astype(int)

    # .500 line increment (ties = 0)
    long["delta"]   = long["is_win"] - long["is_loss"]

    # 6) Order & per-team sequencing ------------------------------------------
    long = long.sort_values(["team_id","Date","Game_ID"], kind="mergesort")
    long["gp"] = long.groupby("team_id").cumcount() + 1

    # 7) Cumulative record & games-above-.500 ----------------------------------
    long["cum_w"] = long.groupby("team_id")["is_win"].cumsum()
    long["cum_l"] = long.groupby("team_id")["is_loss"].cumsum()
    long["cum_t"] = long.groupby("team_id")["is_tie"].cumsum()

    # This is the Y-value for the chart
    long["games_above_500"] = long["cum_w"] - long["cum_l"]

    # --- Goal Differential (GD) -----------------------------------------------
    # Per-game goal differential from the *team's* perspective.
    # For shootouts: most NCAA datasets store regulation+OT goals only (SO adds no goals),
    # so this calculation should already be correct. If your table encodes SO winner goals,
    # strip those first.
    long["gd"] = (long["team_score"].astype(int) - long["opp_score"].astype(int))

    # Cumulative all-time GD: this is the Y-value for a GD curve (step-like, changes only on game days)
    long["cum_gd"] = long.groupby("team_id")["gd"].cumsum()


    timeline = long[[
        "team_id","team","Date","Game_ID","Season_Year","gp",
        "team_score","opp_score",          # (optional but handy)
        "result","delta",                  # win/loss increment (for .500 curve)
        "gd","cum_gd",                     # <-- NEW: per-game and cumulative GD
        "cum_w","cum_l","cum_t","games_above_500",
        "venue"
    ]].copy()

    # timeline = long[[
    #     "team_id","team","Date","Game_ID","Season_Year","gp",
    #     "result","delta","cum_w","cum_l","cum_t","games_above_500","venue"
    # ]].copy()

    # 8) Per-team milestones (for annotations) --------------------------------
    def summarize_team(g: pd.DataFrame) -> pd.Series:
        g = g.sort_values("Date")
        at500 = g[g["games_above_500"] == 0]

        # Locate max/min cumulative GD rows
        max_gd_idx = g["cum_gd"].idxmax()
        min_gd_idx = g["cum_gd"].idxmin()

        return pd.Series({
            "team": g["team"].iloc[0],
            "first_date": g["Date"].min(),
            "last_date":  g["Date"].max(),

            # .500 curve milestones (you already had these)
            "max_point":  g["games_above_500"].max(),
            "max_date":   g.loc[g["games_above_500"].idxmax(),"Date"],
            "min_point":  g["games_above_500"].min(),
            "min_date":   g.loc[g["games_above_500"].idxmin(),"Date"],
            "final_w":    g["cum_w"].iloc[-1],
            "final_l":    g["cum_l"].iloc[-1],
            "final_t":    g["cum_t"].iloc[-1],
            "final_games_above_500": g["games_above_500"].iloc[-1],
            "last_500_date": at500["Date"].iloc[-1] if not at500.empty else pd.NaT,

            # --- NEW: GD milestones
            "max_cum_gd": g.loc[max_gd_idx, "cum_gd"],
            "max_cum_gd_date": g.loc[max_gd_idx, "Date"],
            "min_cum_gd": g.loc[min_gd_idx, "cum_gd"],
            "min_cum_gd_date": g.loc[min_gd_idx, "Date"],
            "final_cum_gd": g["cum_gd"].iloc[-1],
        })

    # def summarize_team(g: pd.DataFrame) -> pd.Series:
    #     g = g.sort_values("Date")
    #     at500 = g[g["games_above_500"] == 0]
    #     return pd.Series({
    #         "team": g["team"].iloc[0],
    #         "first_date": g["Date"].min(),
    #         "last_date":  g["Date"].max(),
    #         "max_point":  g["games_above_500"].max(),
    #         "max_date":   g.loc[g["games_above_500"].idxmax(),"Date"],
    #         "min_point":  g["games_above_500"].min(),
    #         "min_date":   g.loc[g["games_above_500"].idxmin(),"Date"],
    #         "final_w":    g["cum_w"].iloc[-1],
    #         "final_l":    g["cum_l"].iloc[-1],
    #         "final_t":    g["cum_t"].iloc[-1],
    #         "final_games_above_500": g["games_above_500"].iloc[-1],
    #         "last_500_date": at500["Date"].iloc[-1] if not at500.empty else pd.NaT,
    #     })

    summary = timeline.groupby("team_id", group_keys=False).apply(summarize_team).reset_index()

    # 9) Save outputs (CSV by default; Parquet lines are included if you have pyarrow) ----
    timeline.to_csv(out_path / "../TEMP/team_timeseries.csv", index=False)
    summary.to_csv(out_path / "../TEMP/team_summary.csv", index=False)

    # If you prefer Parquet (faster; needs pyarrow or fastparquet):
    # timeline.to_parquet(out_path / "team_timeseries.parquet", index=False)
    # summary.to_parquet(out_path / "team_summary.parquet", index=False)


# ---------- Optional: simple plotting helper ---------------------------------
# (Keeps colors/style up to you; this just draws a step-like line.)
def plot_team_curve(timeline: pd.DataFrame, team_name_or_id):
    """
    Example usage:
        tl = pd.read_csv("team_timeseries.csv", parse_dates=["Date"])
        plot_team_curve(tl, team_name_or_id=123456789)  # or a team name string

    This produces a basic line of `games_above_500` vs. date.
    Add your own shading/annotations to mimic the NHL-style chart.
    """
    import matplotlib.pyplot as plt

    if isinstance(team_name_or_id, (int, np.integer)):
        g = timeline[timeline["team_id"] == team_name_or_id].copy()
        label = str(team_name_or_id)
    else:
        g = timeline[timeline["team"].str.casefold() == str(team_name_or_id).casefold()].copy()
        label = str(team_name_or_id)

    g = g.sort_values("Date")
    plt.figure(figsize=(14,4))
    plt.plot(g["Date"], g["games_above_500"], linewidth=1.25, label=label)
    plt.axhline(0, linewidth=1)
    plt.xlabel("Date"); plt.ylabel("Games Above/Below .500")
    plt.title(f"{g['team'].iloc[0] if not g.empty else label}: Historical Performance")
    plt.legend()
    plt.tight_layout()
    plt.show()


# # ---------- Run from a notebook/script ---------------------------------------
# if __name__ == "__main__":
#     build_timeseries(DATA_PATH)


In [39]:
# Create a plot of a team

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Reload & filter (idempotent)
tl = pd.read_csv(Path("../TEMP/team_summary.csv"), parse_dates=["Date"])
team_name = "Providence"
g = tl[tl["team"].str.casefold() == team_name.casefold()].copy().sort_values("Date")

# Convert to numpy-friendly arrays
x_vals = g["Date"].values
y_vals = g["games_above_500"].astype(float).values  # ensure numeric dtype

# Milestones
max_idx = int(np.nanargmax(y_vals))
min_idx = int(np.nanargmin(y_vals))
last_500 = g[g["games_above_500"] == 0]["Date"].max()

final_w = int(g["cum_w"].iloc[-1])
final_l = int(g["cum_l"].iloc[-1])
final_t = int(g["cum_t"].iloc[-1])

plt.figure(figsize=(14,5))
plt.step(x_vals, y_vals, where="post", linewidth=1.5, label=team_name)

# Shade above/below zero using default colors
plt.fill_between(x_vals, 0.0, y_vals, where=y_vals>0, alpha=0.2, step="post")
plt.fill_between(x_vals, 0.0, y_vals, where=y_vals<0, alpha=0.2, step="post")

plt.axhline(0.0, linewidth=1)

# Annotations
plt.plot(x_vals[max_idx], y_vals[max_idx], marker="o")
plt.annotate(
    f"Highest: {int(y_vals[max_idx])}\n{pd.to_datetime(x_vals[max_idx]).date()}",
    (x_vals[max_idx], y_vals[max_idx]), xytext=(10, 10), textcoords="offset points"
)
plt.plot(x_vals[min_idx], y_vals[min_idx], marker="o")
plt.annotate(
    f"Lowest: {int(y_vals[min_idx])}\n{pd.to_datetime(x_vals[min_idx]).date()}",
    (x_vals[min_idx], y_vals[min_idx]), xytext=(10, -20), textcoords="offset points"
)
if pd.notna(last_500):
    last_val = float(g.loc[g["Date"] == last_500, "games_above_500"].iloc[-1])
    plt.plot(last_500, last_val, marker="o")
    plt.annotate(f"Last .500: {last_500.date()}", (last_500, last_val),
                 xytext=(10, 10), textcoords="offset points")

plt.title(f"{team_name}: Games Above/Below .500 Over Time")
plt.xlabel("Date")
plt.ylabel("Games Above/Below .500")

final_str = f"Final Record: {final_w}-{final_l}-{final_t}  (Δ {final_w - final_l})"
plt.gcf().text(0.99, 0.02, final_str, ha="right", va="bottom", bbox=dict(boxstyle="round,pad=0.3"))

plt.tight_layout()
plt.show()




ValueError: Missing column provided to 'parse_dates': 'Date'

## All Time Goal Differential

In [ ]:
# # --- Goal Differential (GD) -----------------------------------------------
# # Per-game goal differential from the *team's* perspective.
# # For shootouts: most NCAA datasets store regulation+OT goals only (SO adds no goals),
# # so this calculation should already be correct. If your table encodes SO winner goals,
# # strip those first.
# long["gd"] = (long["team_score"].astype(int) - long["opp_score"].astype(int))

# # Cumulative all-time GD: this is the Y-value for a GD curve (step-like, changes only on game days)
# long["cum_gd"] = long.groupby("team_id")["gd"].cumsum()


# timeline = long[[
#     "team_id","team","Date","Game_ID","Season_Year","gp",
#     "team_score","opp_score",          # (optional but handy)
#     "result","delta",                  # win/loss increment (for .500 curve)
#     "gd","cum_gd",                     # <-- NEW: per-game and cumulative GD
#     "cum_w","cum_l","cum_t","games_above_500",
#     "venue"
# ]].copy()


In [ ]:
# def summarize_team(g: pd.DataFrame) -> pd.Series:
#     g = g.sort_values("Date")
#     at500 = g[g["games_above_500"] == 0]

#     # Locate max/min cumulative GD rows
#     max_gd_idx = g["cum_gd"].idxmax()
#     min_gd_idx = g["cum_gd"].idxmin()

#     return pd.Series({
#         "team": g["team"].iloc[0],
#         "first_date": g["Date"].min(),
#         "last_date":  g["Date"].max(),

#         # .500 curve milestones (you already had these)
#         "max_point":  g["games_above_500"].max(),
#         "max_date":   g.loc[g["games_above_500"].idxmax(),"Date"],
#         "min_point":  g["games_above_500"].min(),
#         "min_date":   g.loc[g["games_above_500"].idxmin(),"Date"],
#         "final_w":    g["cum_w"].iloc[-1],
#         "final_l":    g["cum_l"].iloc[-1],
#         "final_t":    g["cum_t"].iloc[-1],
#         "final_games_above_500": g["games_above_500"].iloc[-1],
#         "last_500_date": at500["Date"].iloc[-1] if not at500.empty else pd.NaT,

#         # --- NEW: GD milestones
#         "max_cum_gd": g.loc[max_gd_idx, "cum_gd"],
#         "max_cum_gd_date": g.loc[max_gd_idx, "Date"],
#         "min_cum_gd": g.loc[min_gd_idx, "cum_gd"],
#         "min_cum_gd_date": g.loc[min_gd_idx, "Date"],
#         "final_cum_gd": g["cum_gd"].iloc[-1],
#     })


In [ ]:
def plot_team_goal_diff_styled(timeline, team_name_or_id,
                               era_marks=(1970, 1980, 1990, 2000, 2010, 2020, 2030),
                               line_color="#E6A000",       # warm golden line
                               fill_above="#FFD98F",       # light gold
                               fill_below="#FFD98F",       # same fill; tweak if you want 2-tone
                               figsize=(16,6)):
    """
    Draw a 'cumulative goal differential' chart styled like the .500 graphic.
    Requires timeline to include: Date, cum_gd, cum_w, cum_l, cum_t.

    Example:
        tl = pd.read_csv("team_timeseries.csv", parse_dates=["Date"])
        plot_team_goal_diff_styled(tl, "Michigan State")
    """
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt

    if "cum_gd" not in timeline.columns:
        raise ValueError("cum_gd missing from timeline. Re-run transform with GD patches.")

    # Select team
    if isinstance(team_name_or_id, (int, np.integer)):
        g = timeline[timeline["team_id"] == team_name_or_id].copy()
        label = str(team_name_or_id)
    else:
        g = timeline[timeline["team"].str.casefold() == str(team_name_or_id).casefold()].copy()
        label = str(team_name_or_id)

    if g.empty:
        raise ValueError(f"No rows for team '{team_name_or_id}'")

    g = g.sort_values("Date").reset_index(drop=True)
    x = g["Date"].values
    y = g["cum_gd"].astype(float).values

    # Milestones
    max_idx = int(np.nanargmax(y))
    min_idx = int(np.nanargmin(y))
    last_zero = g.loc[g["cum_gd"] == 0, "Date"].max()  # most recent date at zero GD (if any)

    # Final record & GD
    final_w = int(g["cum_w"].iloc[-1]); final_l = int(g["cum_l"].iloc[-1]); final_t = int(g["cum_t"].iloc[-1])
    final_gd = int(g["cum_gd"].iloc[-1])

    # Plot
    fig, ax = plt.subplots(figsize=figsize)
    ax.step(x, y, where="post", lw=2.2, color=line_color)
    ax.fill_between(x, 0, y, where=y>0, step="post", alpha=0.28, color=fill_above)
    ax.fill_between(x, 0, y, where=y<0, step="post", alpha=0.18, color=fill_below)
    ax.axhline(0, lw=1.2, color="0.25")

    # Era/decade markers
    for yr in era_marks:
        ax.axvline(pd.Timestamp(f"{yr}-01-01"), ls="--", lw=0.8, color="0.65")

    # Annotations: highest/lowest & last-zero-GD
    ax.plot(g["Date"].iloc[max_idx], y[max_idx], "o", ms=5, color=line_color)
    ax.annotate(f"Highest GD: {int(y[max_idx])}\n{pd.to_datetime(x[max_idx]).date()}",
                (g["Date"].iloc[max_idx], y[max_idx]), xytext=(10, 10),
                textcoords="offset points")

    ax.plot(g["Date"].iloc[min_idx], y[min_idx], "o", ms=5, color=line_color)
    ax.annotate(f"Lowest GD: {int(y[min_idx])}\n{pd.to_datetime(x[min_idx]).date()}",
                (g["Date"].iloc[min_idx], y[min_idx]), xytext=(10, -22),
                textcoords="offset points")

    if pd.notna(last_zero):
        last_val = float(g.loc[g["Date"] == last_zero, "cum_gd"].iloc[-1])
        ax.plot(last_zero, last_val, "o", ms=5, color="tab:purple")
        ax.annotate(f"Last 0 GD: {last_zero.date()}",
                    (last_zero, last_val), xytext=(10, 10),
                    textcoords="offset points")

    # Title & labels
    title = g["team"].iloc[0]
    ax.set_title(f"{title}: Cumulative Goal Differential Over Time", pad=10)
    ax.set_ylabel("Cumulative Goal Differential")
    ax.set_xlabel("Date")
    ax.margins(x=0.01)

    # Footer badge
    footer = f"Final Record: {final_w}-{final_l}-{final_t}   Final GD: {final_gd:+d}"
    fig.text(0.99, 0.02, footer, ha="right", va="bottom",
             bbox=dict(boxstyle="round,pad=0.35", fc="#FFE7B8", ec=line_color, lw=1.2))

    fig.tight_layout()
    plt.show()

### Call Goal Differential Plot
- set tl = to team_summary table
- pass tl and Team name string to plot_team_goal_diff_styled

In [ ]:
tl = pd.read_csv("../data/tables/team_summary.csv", parse_dates=["Date"])

if "cum_gd" not in tl.columns:
    if {"team_score", "opp_score"}.issubset(tl.columns):
        tl["gd"] = tl["team_score"].astype("int64") - tl["opp_score"].astype("int64")
        tl["cum_gd"] = tl.groupby("team_id")["gd"].cumsum()
        tl.to_csv(TIMELINE_CSV, index=False)
        print("Patched timeline with gd/cum_gd and saved.")
    else:
        raise ValueError(
            "Timeline missing cum_gd and also lacks team_score/opp_score. "
            "Re-run the transform so those columns are included."
        )
else:
    print("Timeline already has cum_gd.")
# ✅ Use the timeseries, not the summary
# tl = pd.read_csv("../TEMP/team_timeseries.csv", parse_dates=["Date"])

# sanity check
# assert "cum_gd" in tl.columns and "Date" in tl.columns


In [ ]:

# plot
plot_team_goal_diff_styled(tl, "Michigan State")
